In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context, getNode
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10c820c10>: 130, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 100
# children: 10


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            for f in footnotes:
                displayMarkdown('1. %s' % f)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 1196.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())
    
# Note misleading cum times below due to recursion...

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 2216.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:items|121|11,055,961|852|91,371|GetValue
|Portfolio:items|121|11,055,109|18,563|91,364|GetValue/Calc
|Report:data|1|5,766,710|11|5,766,710|GetValue
|Report:data|1|5,766,698|411|5,766,698|GetValue/Calc
|TradingContainer:NPV|11|5,759,618|103|523,601|GetValue
|TradingContainer:NPV|11|5,759,514|1,605|523,592|GetValue/Calc
|Workbook:items|1,104|4,533,762|7,912|4,106|GetValue
|Workbook:items|1,100|4,525,849|4,395,035|4,114|GetValue/Calc
|Root:Clocks|2|1,594,384|2,624|797,192|Context
|Portfolio:children|242|1,028,084|1,069|4,248|GetValue
|Portfolio:children|121|1,027,015|514,928|8,487|GetValue/Calc
|Root:End|1|480,982|45|480,982|Context
|Root:Start|1|472,190|16|472,190|Context
|Root:Amend MarketData|1|462,615|35|462,615|Context
|Root:Activity Portfolio|1|462,519|40|462,519|Context
|Root:Activity Trading|1|462,214|38|462,214|Context
|Root:Amend Trading|1|460,434|39|460,434|Context
|Root:Activity MarketData|1|457,784|41|457,784|Context
|Root:Amend Portfolio|1|455,196|33|455,196|Context
|Root:Start breaks|1|454,246|35|454,246|Context
|PortfolioUpdateEvent:children|121|433,524|2,213|3,582|GetValue
|TradingBook|102|400,235|400,235|3,923|Db.Get
|Equity:NPV|15|171,478|149|11,431|GetValue
|Equity:NPV|15|171,329|358|11,421|GetValue/Calc
|MarketInterface:spot|15|159,068|98|10,604|GetValue
|MarketInterface:spot|15|158,969|383|10,597|GetValue/Calc
|ExternalRefData:state|15|151,038|87|10,069|GetValue
|ExternalRefData:state|15|150,951|251|10,063|GetValue/Calc
|RefData:state|15|150,700|95|10,046|GetValue
|RefData:state|15|150,604|64,049|10,040|GetValue/Calc
|Clock:cutoffs|2,480|135,632|5,577|54|GetValue
|Clock:cutoffs|20|130,054|366|6,502|GetValue/Calc
|Clock:parent|20|97,338|117|4,866|GetValue
|Clock:parent|20|97,220|81,261|4,861|GetValue/Calc
|PortfolioUpdateEvent|12|54,673|54,673|4,556|Db.Get
|RefDataUpdateEvent|9|41,104|41,104|4,567|Db.Get
|TradingPortfolio|10|38,458|38,458|3,845|Db.Get
|TradingBook:clock|2,200|32,261|14,377|14|GetValue
|Clock|5|19,204|19,204|3,840|Db.Get
|TradingBook:clock|1,100|17,884|13,768|16|GetValue/Calc
|TradeOpenEvent|4|17,218|17,218|4,304|Db.Get
|TradeOpenEvent:ticket|66|14,912|465|225|GetValue
|TradingTicket|4|14,446|14,446|3,611|Db.Get
|Equity:refdata|15|11,902|163|793|GetValue
|Equity:refdata|15|11,739|361|782|GetValue/Calc
|_WorkItemEvent:item|33|7,895|185|239|GetValue
|ClockEvent|2|7,824|7,824|3,912|Db.Get
|Equity|2|7,710|7,710|3,855|Db.Get
|_WorkItemEvent:book2|66|7,627|243|115|GetValue
|ClockEvent:parent|8|7,574|60|946|GetValue
|MarketInterface:source|15|7,547|281|503|GetValue
|MarketInterface|2|7,337|7,337|3,668|Db.Get
|MarketInterface:source|15|7,265|319|484|GetValue/Calc
|Portfolio:clock|242|7,183|1,722|29|GetValue
|MarketDataSource|2|6,811|6,811|3,405|Db.Get
|Portfolio:books|231|5,526|1,451|23|GetValue
|Portfolio:clock|121|5,460|1,485|45|GetValue/Calc
|Portfolio:books|121|4,075|3,579|33|GetValue/Calc
|MarketDataSource:clock|30|3,979|201|132|GetValue
|MarketInterface|1|3,968|3,968|3,968|Db.Put
|MarketDataSource:clock|15|3,777|178|251|GetValue/Calc
|TradeOpenEvent:premium|33|3,745|170|113|GetValue
|RootClock|1|3,707|3,707|3,707|Db.Get
|ForwardCashflow|1|3,575|3,575|3,575|Db.Get
|Event:amends|230|1,176|1,176|5|GetValue
|Entity:clock|40|426|256|10|GetValue
|_WorkItemEvent:book1|66|244|244|3|GetValue
|TradeOpenEvent:quantity|66|202|202|3|GetValue
|RefDataUpdateEvent:data|58|195|195|3|GetValue
|Entity:clock|20|169|169|8|GetValue/Calc
|ForwardCashflow:NPV|11|136|108|12|GetValue
|MarketInterface:sourceName|15|134|98|8|GetValue
|TradeOpenEvent:action|33|129|129|3|GetValue
|TradeOpenEvent:unitPrice|33|119|119|3|GetValue
|RootClock:cutoffs|52|109|109|2|GetValue
|Equity:assetName|15|72|72|4|GetValue
|MarketInterface:sourceName|15|36|36|2|GetValue/Calc
|ForwardCashflow:NPV|11|27|27|2|GetValue/Calc
|Report:valuable|1|4|4|4|GetValue
|Report:ts2|1|2|2|2|GetValue
|Report:ts1|1|2|2|2|GetValue